In [16]:
import pandas as pd
import numpy as np
import seaborn as sns

# Import modules from Scikit-learn
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split   # Import train_test_split function
from sklearn import metrics   # import metrics modules for accuracy calculation
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import time
from tqdm import tqdm

In [17]:
# Read data
PATH = "../../my_data/identification-dataset/my_custom_data/anblock-error-dataset.csv"
df = pd.read_csv(PATH)

# Drop uncomplete rows
df.dropna(inplace=True)

In [18]:
# Set training data
train_df = df.drop('material', axis=1)

# Extracted features 
X = train_df.drop('encoded_material', axis=1)
y = train_df['encoded_material'] # Labels

In [19]:
# Feature Scaling
# scaler = MinMaxScaler()
# X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [20]:
# Split dataset into training and test set
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3) # 70% training and 30% test

In [21]:
# different variations of hidden layer combinations
hidden_layers_50 = [(50,), (50, 50), (50, 50, 50), (50, 50, 50, 50), (50, 50, 50, 50, 50), (50, 50, 50, 50, 50, 50)]
hidden_layers_100 = [(100,), (100, 100), (100, 100, 100), (100, 100, 100, 100), (100, 100, 100, 100, 100), (100, 100, 100, 100, 100, 100)]

In [22]:
# Create an empty list to store results
def trainer(hidden_layer_sizes_list, X, y):
    # Randomly split dataset into training and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3) # 70% training and 30% test

    results = []
    for sizes in hidden_layer_sizes_list:
        # Create a pipeline object for the model
        pipe_MLP = make_pipeline(StandardScaler(),
                                MLPClassifier(solver='adam',
                                            activation='relu',
                                            
                                            hidden_layer_sizes=sizes,
                                            random_state=0,
                                            max_iter=500           # TODO: tune it later
                                            # verbose=True
                                            )
                                )
        
        # Measure training time
        start_train = time.time()
        pipe_MLP.fit(X_train, y_train)
        end_train = time.time()
        train_time_per_sample = (end_train - start_train) / len(X_train)

        # Measure test time
        start_test = time.time()
        y_pred = pipe_MLP.predict(X_test)
        end_test = time.time()
        test_time_per_sample = (end_test - start_test) / len(X_test)
        
        # Evaluate the pipeline and store the results
        accuracy = metrics.accuracy_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred, average="macro")
        recall = metrics.recall_score(y_test, y_pred, average="macro")
        f1 = metrics.recall_score(y_test, y_pred, average="macro")


        results.append({
            'hidden_layer_sizes': sizes,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'train_time_per_sample': train_time_per_sample,
            'test_time_per_sample': test_time_per_sample 
        })
    return results

In [23]:
results = []
for x in tqdm(np.arange(10)):
    r = trainer(hidden_layers_50, X, y)
    results.append(r)

  0%|          | 0/10 [00:00<?, ?it/s]/Users/bentmeemken/miniconda3/envs/dataScience/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
 10%|█         | 1/10 [03:21<30:16, 201.82s/it]/Users/bentmeemken/miniconda3/envs/dataScience/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
 20%|██        | 2/10 [06:36<26:21, 197.66s/it]/Users/bentmeemken/miniconda3/envs/dataScience/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
 30%|███       | 3/10 [10:02<23:31, 201.59s/it]/Users/bentmeemken/miniconda3

In [24]:
# Flatten the list of lists
flattened_results = [item for sublist in results for item in sublist]

# Convert to DataFrame
flattend_results_df = pd.DataFrame(flattened_results)

In [33]:
mean_df = flattend_results_df.groupby('hidden_layer_sizes').mean().reset_index()
mean_df

,hidden_layer_sizes,accuracy,precision,recall,f1,train_time_per_sample,test_time_per_sample
0,"(50,)",0.982067,0.982167,0.982139,0.982139,0.010180,0.000002
1,"(50, 50)",0.985594,0.985836,0.985663,0.985663,0.003608,0.000001
2,"(50, 50, 50)",0.983478,0.983905,0.983491,0.983491,0.000826,0.000002
3,"(50, 50, 50, 50)",0.975539,0.976049,0.975594,0.975594,0.001121,0.000003
4,"(50, 50, 50, 50, 50)",0.977560,0.977782,0.977687,0.977687,0.001033,0.000004
5,"(50, 50, 50, 50, 50, 50)",0.967516,0.968161,0.967553,0.967553,0.005474,0.000005


In [34]:
# Round the columns to the 4th decimal place
mean_df['accuracy'] = mean_df['accuracy'].round(2)
mean_df['precision'] = mean_df['precision'].round(2)
mean_df['recall'] = mean_df['recall'].round(2)
mean_df['f1'] = mean_df['f1'].round(2)

# Scale from seconds to milliseconds
mean_df['train_time_per_sample'] = mean_df['train_time_per_sample'] * 1000 # to millisec. 
mean_df['train_time_per_sample'] = mean_df['train_time_per_sample'].round(2)

mean_df['test_time_per_sample'] = mean_df['test_time_per_sample'] * 1000000 # to microsec.
mean_df['test_time_per_sample'] = mean_df['test_time_per_sample'].round(2)

mean_df

,hidden_layer_sizes,accuracy,precision,recall,f1,train_time_per_sample,test_time_per_sample
0,"(50,)",0.98,0.98,0.98,0.98,10.18,1.57
1,"(50, 50)",0.99,0.99,0.99,0.99,3.61,1.06
2,"(50, 50, 50)",0.98,0.98,0.98,0.98,0.83,1.78
3,"(50, 50, 50, 50)",0.98,0.98,0.98,0.98,1.12,2.85
4,"(50, 50, 50, 50, 50)",0.98,0.98,0.98,0.98,1.03,3.94
5,"(50, 50, 50, 50, 50, 50)",0.97,0.97,0.97,0.97,5.47,5.00


In [36]:
std_df = flattend_results_df.groupby('hidden_layer_sizes').std().reset_index()
std_df

,hidden_layer_sizes,accuracy,precision,recall,f1,train_time_per_sample,test_time_per_sample
0,"(50,)",0.001694,0.001726,0.001756,0.001756,0.024501,2.039332e-06
1,"(50, 50)",0.005234,0.004946,0.005192,0.005192,0.008541,2.621840e-07
2,"(50, 50, 50)",0.006858,0.006197,0.006917,0.006917,0.000424,6.378611e-07
3,"(50, 50, 50, 50)",0.008223,0.007719,0.008146,0.008146,0.001326,4.390778e-07
4,"(50, 50, 50, 50, 50)",0.004750,0.004519,0.004648,0.004648,0.000673,1.602857e-06
5,"(50, 50, 50, 50, 50, 50)",0.025613,0.024705,0.025694,0.025694,0.013350,1.221350e-06


In [37]:
# Round the columns to the 4th decimal place
std_df['accuracy'] = std_df['accuracy'].round(4)
std_df['precision'] = std_df['precision'].round(4)
std_df['recall'] = std_df['recall'].round(4)
std_df['f1'] = std_df['f1'].round(4)

# Scale from seconds to milliseconds
std_df['train_time_per_sample'] = std_df['train_time_per_sample'] * 1000 # to millisec. 
std_df['train_time_per_sample'] = std_df['train_time_per_sample'].round(2)

std_df['test_time_per_sample'] = std_df['test_time_per_sample'] * 1000000 # to microsec.
std_df['test_time_per_sample'] = std_df['test_time_per_sample'].round(2)

std_df

,hidden_layer_sizes,accuracy,precision,recall,f1,train_time_per_sample,test_time_per_sample
0,"(50,)",0.0017,0.0017,0.0018,0.0018,24.50,2.04
1,"(50, 50)",0.0052,0.0049,0.0052,0.0052,8.54,0.26
2,"(50, 50, 50)",0.0069,0.0062,0.0069,0.0069,0.42,0.64
3,"(50, 50, 50, 50)",0.0082,0.0077,0.0081,0.0081,1.33,0.44
4,"(50, 50, 50, 50, 50)",0.0047,0.0045,0.0046,0.0046,0.67,1.60
5,"(50, 50, 50, 50, 50, 50)",0.0256,0.0247,0.0257,0.0257,13.35,1.22


In [28]:
markdown_table = mean_df.to_markdown(index=False)
markdown_table

'| hidden_layer_sizes       |   accuracy |   precision |   recall |   f1 |   train_time_per_sample |   test_time_per_sample |\n|:-------------------------|-----------:|------------:|---------:|-----:|------------------------:|-----------------------:|\n| (50,)                    |       0.98 |        0.98 |     0.98 | 0.98 |                10179.6  |                   1.57 |\n| (50, 50)                 |       0.99 |        0.99 |     0.99 | 0.99 |                 3608.27 |                   1.06 |\n| (50, 50, 50)             |       0.98 |        0.98 |     0.98 | 0.98 |                  825.96 |                   1.78 |\n| (50, 50, 50, 50)         |       0.98 |        0.98 |     0.98 | 0.98 |                 1121.14 |                   2.85 |\n| (50, 50, 50, 50, 50)     |       0.98 |        0.98 |     0.98 | 0.98 |                 1033.02 |                   3.94 |\n| (50, 50, 50, 50, 50, 50) |       0.97 |        0.97 |     0.97 | 0.97 |                 5474.38 |                  

| hidden_layer_sizes       |   accuracy |   precision |   recall |       f1 |   train_time_per_sample |   test_time_per_sample |
|:-------------------------|-----------:|------------:|---------:|---------:|------------------------:|-----------------------:|
| (50,)                    |   0.98186  |    0.982025 | 0.982028 | 0.982028 |             0.00139544  |            5.787e-07   |
| (50, 50)                 |   0.991154 |    0.99135  | 0.991123 | 0.991123 |             0.00128966  |            1.33319e-06 |
| (50, 50, 50)             |   0.988019 |    0.988097 | 0.98822  | 0.98822  |             0.000961179 |            1.56593e-06 |
| (50, 50, 50, 50)         |   0.98046  |    0.981017 | 0.980441 | 0.980441 |             0.000818557 |            2.89977e-06 |
| (50, 50, 50, 50, 50)     |   0.980404 |    0.980523 | 0.980583 | 0.980583 |             0.000880122 |            3.19842e-06 |
| (50, 50, 50, 50, 50, 50) |   0.983875 |    0.98393  | 0.984097 | 0.984097 |             0.00145     |            4.61586e-06 |

In [29]:
# Convert DataFrame to LaTeX table
latex_table = mean_df.to_latex(index=False)
print(latex_table)

\begin{tabular}{lrrrrrr}
\toprule
      hidden\_layer\_sizes &  accuracy &  precision &  recall &   f1 &  train\_time\_per\_sample &  test\_time\_per\_sample \\
\midrule
                   (50,) &      0.98 &       0.98 &    0.98 & 0.98 &               10179.58 &                  1.57 \\
                (50, 50) &      0.99 &       0.99 &    0.99 & 0.99 &                3608.27 &                  1.06 \\
            (50, 50, 50) &      0.98 &       0.98 &    0.98 & 0.98 &                 825.96 &                  1.78 \\
        (50, 50, 50, 50) &      0.98 &       0.98 &    0.98 & 0.98 &                1121.14 &                  2.85 \\
    (50, 50, 50, 50, 50) &      0.98 &       0.98 &    0.98 & 0.98 &                1033.02 &                  3.94 \\
(50, 50, 50, 50, 50, 50) &      0.97 &       0.97 &    0.97 & 0.97 &                5474.38 &                  5.00 \\
\bottomrule
\end{tabular}



/var/folders/7l/08clpl7s5sg024cxlpgy5vtm0000gn/T/ipykernel_11453/1483047706.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = mean_df.to_latex(index=False)
